## 사용된 blockchain data는 아래의 github 코드를 사용한 데이터 베이스를 사용함
- LuHa(munhyunsu)
    - Repository : BitcoinAnalysis
    - URL : https://github.com/munhyunsu/BitcoinAnalysis

In [ ]:
import sqlite3
import time

# 62만 블록이 저장된 데이터 베이스 사용
conn = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/index_620000.db")
cur = conn.cursor()
tmp_conn = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/index_620000.db")
tmp_cur = tmp_conn.cursor()

# database의 table확인하기
cur.execute('SELECT name from sqlite_master where type= "table"')
print(cur.fetchall())

### RootAddress를 가진 엔티티 csv파일을 가져와 사용

In [2]:
import pandas as pd

path = './ChainAnalysisNamedClusters.csv'
category_addr = pd.read_csv(path)
category_addr = category_addr[['Category', 'RootAddress']]
category_addr.head()

,Category,RootAddress
0,darknet market,17Bccs6eJAKsXHeopcH5FCt7o1FdCk1Xw7
1,gambling,1rH4vGpf5LEgCvUxXgRycGrQ5ahsxXB2F
2,scam,1Kd9zuLiJ5mbz6YD7a8nQiiAXvPuveUyRL
3,exchange,14zRXp2AZ8yxXzPXJ5gAirmgAxJdUZiVut
4,mining pool,1F1xcRt8H8Wa623KqmkEontwAAVqDSAWCV


In [3]:
category_addr['Category'].value_counts()

exchange                426
gambling                112
darknet market           82
mining pool              80
other                    72
high risk exchange       63
scam                     38
ransomware               28
stolen funds             22
merchant services        21
mixing                   20
hosted wallet            18
p2p exchange              6
terrorist financing       5
sanctions                 4
child abuse material      3
mining                    1
Name: Category, dtype: int64

### 특정 엔티티를 선정

In [4]:
select_category_addr = category_addr[category_addr['Category'].isin(['darknet market', 'exchange', 'gambling', 'high risk exchange', 'hosted wallet', 'merchant services', 'mining pool', 'mixing', 'ransomware', 'scam', 'stolen funds', 'other'])]
select_category_addr.head()

,Category,RootAddress
0,darknet market,17Bccs6eJAKsXHeopcH5FCt7o1FdCk1Xw7
1,gambling,1rH4vGpf5LEgCvUxXgRycGrQ5ahsxXB2F
2,scam,1Kd9zuLiJ5mbz6YD7a8nQiiAXvPuveUyRL
3,exchange,14zRXp2AZ8yxXzPXJ5gAirmgAxJdUZiVut
4,mining pool,1F1xcRt8H8Wa623KqmkEontwAAVqDSAWCV


In [5]:
category_label = list(set(select_category_addr['Category']))
category_per_dict = dict() # 카테고리별 RootAddress들의 모음 list를 담은 dict
for cl in category_label:
    cl_per_addr = set(select_category_addr[select_category_addr['Category'] == cl]['RootAddress'])
    category_per_dict[cl] = cl_per_addr

In [6]:
# category별로 정리를 했으며, 해당 주소들 중 db에 있는 것만 골라서 정리
category_per_addr = dict()
for cpd in category_per_dict:
    addr_list = list()
    for ad in category_per_dict[cpd]:
        cur.execute('''SELECT * from AddrID where addr = ?''', (ad,))
        try:
            addr_list.append((cur.fetchone())[1]) # db에 있는 (index, addr)에서 addr만 추출
        except:
            continue
    category_per_addr[cpd] = addr_list

In [7]:
def apply_multi_input(addr_id):
    addres_for_cluster = set()
    
    cur.execute('''SELECT tx FROM TxIn WHERE addr = ?''', (addr_id,))
    for row in cur:
        if len(addres_for_cluster) == 20: ## 20개로 개수 제한을 두어 전체 multi input 클러스터를 선정하지는 않음
            break
        addres_for_cluster.add(row[0])
        
    return addres_for_cluster

In [8]:
category_multi_input = dict()

for cpa in category_per_addr:
    count = 0
    for addr in category_per_addr[cpa]:
        cur.execute('''select id from AddrID where addr = ?''', (addr,))
        category_multi_input[str(cpa) + str(count)] = apply_multi_input(cur.fetchone()[0])
        count += 1

In [9]:
category_multi_input

{'mining pool0': {138819, 939115},
 'mining pool1': {166738060},
 'mining pool2': {276212796,
  277303483,
  277468986,
  277468987,
  277468992,
  277600137,
  278796259,
  284134666,
  284137930,
  284137951,
  284145290,
  284145291,
  290443535,
  293890493,
  294991137,
  295308124,
  296398402,
  297855038,
  299171495,
  300839912},
 'mining pool3': {198389442,
  198956326,
  200610710,
  200842177,
  200922044,
  201144426,
  201179925,
  201572418,
  201692639,
  202028257,
  202154958,
  202286747,
  202569770,
  202742119,
  203087925,
  203408838,
  203697270,
  203703520,
  203970010,
  204366393},
 'mining pool4': {343888967,
  343891683,
  344985980,
  345404967,
  348017187,
  352070325,
  353119988,
  354314072,
  354549251,
  354553989,
  356452404,
  356753708,
  356753710,
  357121116,
  358122517,
  358163339,
  359249764,
  359691761,
  360263077,
  361278140},
 'mining pool5': {170769029,
  170865439,
  171674572,
  171946484,
  172443233,
  172733474,
  17363475

### input에 존재하는 거래의 id를 모았으니 이제 해당 거래의 모든 input주소들을 하나의 저장소로 모아서 저장

In [10]:
def searched_addresses(tx_id):
    address_set = set()
    
    cur.execute('''SELECT addr FROM TxIn WHERE tx = ?''', (tx_id,))
    for row in cur:
        tmp_cur.execute('''select addr from AddrID where id = ?''', (row[0],))
        for arow in tmp_cur:
            address_set.add(arow[0])
            
    return address_set

In [11]:
category_multi_input_addr = dict()

for cmi in category_multi_input:
    for ctx in category_multi_input[cmi]:
        if ctx == set():
            continue
        category_multi_input_addr[cmi] = searched_addresses(ctx)
len(category_multi_input_addr)

975

## Webcrawler를 진행한 ransomware csv파일 이용

In [12]:
cransom = pd.read_csv('./crawler_ransomware.csv')
cransom_set = set(cransom['Bitcoin Address'].values)

In [13]:
# ransomware 수집했었던 것을 1개로 가져와서 추가
category_multi_input_addr['ransomware28'] = cransom_set

In [ ]:
# # root addr내 주소가 1개인 클러스터들은 제거
# delete_cluster = list()
# for cmia in category_multi_input_addr:
#     if len(category_multi_input_addr[cmia]) == 1:
#         delete_cluster.append(cmia)
#         continue
#     print(cmia + "_length : " + str(len(category_multi_input_addr[cmia])))

# for dc in delete_cluster:
#     del category_multi_input_addr[dc]
# len(category_multi_input_addr)

## 특징 추출 코드

In [14]:
# bitcoin 시세 (2010-01-01 ~ 2020-03-28)
# https://kr.investing.com/crypto/bitcoin/historical-data
import pandas as pd
from datetime import datetime
bitcoin_past_csv = pd.read_csv('./bitcoin_past_value.csv')

def change_date_format(date):
    year = date.split('년 ')
    month = year[1].split('월 ')
    day = month[1].split('일')[0]
    
    return year[0] + '-' + month[0] + '-' + day

bitcoin_past_csv = bitcoin_past_csv[['날짜', '종가']]
bitcoin_past_csv['date'] = bitcoin_past_csv.apply(lambda x:change_date_format(x['날짜']), axis=1)

preprocess_csv = bitcoin_past_csv.copy()
preprocess_csv = preprocess_csv.drop(['날짜'], axis=1)
preprocess_csv = preprocess_csv.rename(columns = {'종가' : 'USD'})
preprocess_csv = preprocess_csv[['date', 'USD']]
preprocess_csv.head()

,date,USD
0,2020-05-22,"9,066.3"
1,2020-05-21,"9,059.0"
2,2020-05-20,"9,512.3"
3,2020-05-19,"9,773.3"
4,2020-05-18,"9,730.7"


### Address BTC Sent / Address USD Sent

In [ ]:
def get_addr_btc_usd_spent(addr_id):
    total_spent_value = 0.0; total_spent_usd = 0.0; convert_susd = 0.0
    tx_cache_index = -1
    txout_rpc_content = list()
    
    # addr id사용해서 txhash.db의 TxOut table에서 id가 일치하는 것 출력
    cur.execute('''SELECT tx, n FROM TxOut WHERE addr = ?''', (addr_id,))
    for row in cur:
        if row[0] != tx_cache_index:
            tx_cache_index = row[0]
            
            tmp_cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (row[0],))
            find_txhash = tmp_cur.fetchone()[0] # searched txhash
            txout_rpc_content = rpc_connection.getrawtransaction(find_txhash, 1)
        
        # 이미 방문했었던 거래의 다른 btc 계산
        now_value = float(txout_rpc_content['vout'][row[1]]['value'])
        total_spent_value += now_value
        
        # usd 계산
        active_time = txout_rpc_content['time']
        itime = datetime.datetime.fromtimestamp(active_time).strftime('%Y-%m-%d') # convert to datetime
        try:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == itime]['USD'])[0].replace(",","")) # search usd
        except IndexError:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == '2010-07-18']['USD'])[0].replace(",","")) # search usd
        total_spent_usd += now_value * convert_susd
        
    return total_spent_value, total_spent_usd

### Address BTC Received / Address USD Received / Address Balance

In [16]:
def get_addr_btc_usd_received(addr_id):
    total_received_value = 0.0 # btc
    total_received_usd = 0.0; convert_susd = 0.0 # usd
    sum_utxo_btc = 0.0 # balance
    
    tx_cache_index = -1 # txhash_index를 저장 -> tx_index
    txin_rpc_content = list() # input에 해당하는 rpc tx 내용 저장 -> [rpc_tx_in_context]

    cur.execute('''SELECT tx, n FROM TxIn WHERE addr = ?''', (addr_id,)) # 약 40만개
    for row in cur:
        try:
            if row[0] != tx_cache_index: # 한번도 방문을 하지 않았던 거래
                tx_cache_index = row[0] # 방문했다는 표시

                tmp_cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (row[0],))
                first_tx_in = tmp_cur.fetchone()[0] # inputs에 해당 주소가 있는 거래의 hash값을 가져옴
                txin_rpc_content = rpc_connection.getrawtransaction(first_tx_in, 1)

            # 방문하지 않았던 거래의 경우 새롭게 값 할당 후, 아래의 코드 실행
            # 방문을 이미 했었던 거래의 경우는 바로 아래의 코드 실행
            txin_to_txout_hash = txin_rpc_content['vin'][row[1]]['txid']
            txin_to_txout_n = txin_rpc_content['vin'][row[1]]['vout']
            
            now_value = float(rpc_connection.getrawtransaction(txin_to_txout_hash, 1)['vout'][txin_to_txout_n]['value'])
            total_received_value += now_value
            
            # usd 계산
            active_time = txin_rpc_content['time']
            itime = datetime.datetime.fromtimestamp(active_time).strftime('%Y-%m-%d') # convert to datetime
            try:
                convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == itime]['USD'])[0].replace(",","")) # search usd
            except IndexError:
                convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == '2010-07-18']['USD'])[0].replace(",","")) # search usd

            total_received_usd += now_value * convert_susd
            # ----------------------------------------------------------------------------------------------------------------------------
            # balance 계산
            for vo in txin_rpc_content['vout']:
                try:
                    tmp_cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (vo['scriptPubKey']['addresses'][0],))
                    addr_index = tmp_cur.fetchone()[0]
                    tmp_cur.execute('''SELECT count(*) FROM TxIn WHERE addr = ?''', (addr_index,))
                    if tmp_cur.fetchone()[0] == 0:
                        sum_utxo_btc += float(vo['value'])
                except KeyError:
                    continue
        except JSONRPCException:
            continue
        except TypeError:
            print(txin_rpc_content)
            continue
            
    return total_received_value, total_received_usd, sum_utxo_btc

### Address lifetime(age)

In [17]:
def get_age(addr_id):
    # txhash를 TxIn, TxOut에서 찾아야 함
    hash_in_list = list(); hash_out_list = list()
    cur.execute('''SELECT tx FROM TxIn WHERE addr = ?''', (addr_id,))
    for row in cur:
        hash_in_list.append(row[0]) # tx가 들어있는 id를 저장 / 중복 제거
    cur.execute('''SELECT tx FROM TxOut WHERE addr = ?''', (addr_id,))
    for row in cur:
        hash_out_list.append(row[0])
    
    if len(hash_in_list) != 0 and len(hash_out_list) != 0:
        check_min_max = [hash_in_list[0], hash_out_list[0], hash_in_list[len(hash_in_list) - 1], hash_out_list[len(hash_out_list) - 1]]
        check_time = list()
        for cmm in check_min_max:
            cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (cmm,))
            txhash = cur.fetchone()[0]
            check_time.append(rpc_connection.getrawtransaction(txhash, 1)['time'])

        # check min active time
        if check_time[0] < check_time[1]:
            check_time.append(check_time[0])
        else:
            check_time.append(check_time[1])

        # check max active time
        if check_time[2] < check_time[3]:
            check_time.append(check_time[3])
        else:
            check_time.append(check_time[2])

        return check_time[5] - check_time[4]
    elif len(hash_in_list) == 0 and len(hash_out_list) > 1:
        cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (hash_out_list[0],))
        txhash = cur.fetchone()[0]
        min_time = rpc_connection.getrawtransaction(txhash, 1)['time']
        cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (hash_out_list[1],))
        txhash = cur.fetchone()[0]
        max_time = rpc_connection.getrawtransaction(txhash, 1)['time']
        return max_time - min_time
    elif len(hash_in_list) > 1 and len(hash_out_list) == 0:
        cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (hash_in_list[0],))
        txhash = cur.fetchone()[0]
        min_time = rpc_connection.getrawtransaction(txhash, 1)['time']
        cur.execute('''SELECT txhash FROM TxID WHERE id = ?''', (hash_in_list[1],))
        txhash = cur.fetchone()[0]
        max_time = rpc_connection.getrawtransaction(txhash, 1)['time']
        return max_time - min_time
    else:
        return 0.0

### Transaction fee

In [18]:
def get_fee_tx(txin_sum_btc, txout_sum_btc):
    return txin_sum_btc - txout_sum_btc

### Address Number Sent

In [19]:
def get_number_spent(addr_id):
#     # AddrID에서 addr의 index먼저 찾기
#     index_cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (saddr,))
#     find_addr_id = index_cur.fetchone()[0] # searched addr index
    # addr id사용해서 txhash.db의 TxOut table에서 id가 일치하는 것 출력
    cur.execute('''SELECT count(*) FROM TxOut WHERE addr = ?''', (addr_id,))
    return cur.fetchone()[0]

### Address Number Received

In [20]:
def get_number_received(addr_id):
#     # AddrID에서 addr의 index먼저 찾기
#     index_cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (saddr,))
#     find_addr_id = index_cur.fetchone()[0] # searched addr index
    # addr id사용해서 txhash.db의 TxOut table에서 id가 일치하는 것 출력
    cur.execute('''SELECT count(*) FROM TxIn WHERE addr = ?''', (addr_id,))
    return cur.fetchone()[0]

# 특징 추출 함수들을 호출하는 컨트롤 함수

In [21]:
from tqdm import tqdm

def max_min_avg_std(flist):
    return (max(flist), min(flist), np.mean(flist)) # 일단 std구하는건 제거한 상태

def control_address_feature(cluster_addresses):
    btc_spent_list = list(); btc_received_list = list()
    usd_spent_list = list(); usd_received_list = list()
    balance_list = list(); age_list = list()
    fee_tx_list = list(); number_spent_list = list(); number_received_list = list()
    
    for ca in tqdm(cluster_addresses):
        # AddrID에서 addr의 index먼저 찾기 (현재 특징의 공통된 작업임)
        cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (ca,))
        find_addr_id = cur.fetchone()[0] # searched addr index
        try:
            spent_btc, spent_usd = get_addr_btc_usd_spent(find_addr_id)
            btc_spent_list.append(spent_btc)
            usd_spent_list.append(spent_usd)
            received_btc, received_usd, balance = get_addr_btc_usd_received(find_addr_id)
            btc_received_list.append(received_btc)
            usd_received_list.append(received_usd)
            balance_list.append(balance)
            age_list.append(get_age(find_addr_id))
            fee_tx_list.append(get_fee_tx(spent_btc, received_btc))
            number_spent_list.append(get_number_spent(find_addr_id))
            number_received_list.append(get_number_received(find_addr_id))
        except JSONRPCException:
            continue
        except OSError:
            continue
    
    try:
        return [max_min_avg_std(btc_spent_list), max_min_avg_std(btc_received_list), max_min_avg_std(usd_spent_list), 
        max_min_avg_std(usd_received_list), max_min_avg_std(balance_list), max_min_avg_std(age_list),
        max_min_avg_std(fee_tx_list), max_min_avg_std(number_spent_list), max_min_avg_std(number_received_list)]
    except ValueError:
        return list()

## Blockchain JSON_RPC이용하기 위한 코드

In [22]:
from secret import rpc_user, rpc_password

In [23]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

Bitcoind PID: 30675



In [24]:
from address_convertor import pubkey_to_address, get_pubkey

In [25]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [26]:
import datetime
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

Best Block Heights: 635911, Time: 2020-06-23T09:53:50+09:00


## 여기가 multi_input으로 클러스터 만든 주소들의 특징 추출과정

In [27]:
len(category_multi_input_addr)

976

In [28]:
import time
import numpy as np

rpc_connection = get_rpc()
cluster_feature = dict()
total_time = time.time()
for cmid in category_multi_input_addr:
    start_time = time.time()
    cluster_feature[cmid] = control_address_feature(category_multi_input_addr[cmid])
    print(cmid + " feature extract finish! - time : " + str(time.time()-start_time))
print('total_time : ' + str(time.time()-total_time))

  0%|          | 0/3 [00:00<?, ?it/s]

mining pool0 feature extract finish! - time : 17.357434034347534


  0%|          | 0/2 [00:00<?, ?it/s]

mining pool1 feature extract finish! - time : 1.026289701461792


  0%|          | 0/4 [00:00<?, ?it/s]

...

gambling4 feature extract finish! - time : 0.2648019790649414


  0%|          | 0/7 [00:00<?, ?it/s]

gambling5 feature extract finish! - time : 0.42307567596435547


 29%|██▊       | 2/7 [00:03<00:08,  1.75s/it]


KeyboardInterrupt: 

In [41]:
# list()만 들어가있는 클러스터 제거(전처리)
new_cluster_feature = dict()
for cf in cluster_feature:
    if cluster_feature[cf] != list():
        new_cluster_feature[cf] = cluster_feature[cf]

In [115]:
import pickle

with open('./category_feature_pickle/mining_scam_exchange_without_four_exchange.pickle', 'wb') as f:
    pickle.dump(cluster_feature, f)

In [112]:
tmp_feature = cluster_feature.copy()

### exchange에서 4대 거래소 제거한 다음 다른 파일의 4대 거래소 특징을 추가하는 부분

## exchange 에서 4대 거래소 제거과정

In [107]:
search_list = ['1BhrNoLRT62o3vdUnXDm62hBpa55goQC8h', '13DdfKxdExuucK3XigjubTrrUWxkEQxYpm', 
               '1E4wTKtmjJN2EMEfYQfTmudyzfDMNnoYCM', '3GDgbaYyoKyY1qc369FhoiqxRCBi9aaHNM']

count = 0
for i, ec in enumerate(category_per_dict['exchange']):
    if count == 4:
        break
    
    if ec in search_list:
        print(str(i) + ' : ' + ec)
        count += 1

58 : 1BhrNoLRT62o3vdUnXDm62hBpa55goQC8h
357 : 3GDgbaYyoKyY1qc369FhoiqxRCBi9aaHNM
368 : 13DdfKxdExuucK3XigjubTrrUWxkEQxYpm
384 : 1E4wTKtmjJN2EMEfYQfTmudyzfDMNnoYCM


In [113]:
print(len(cluster_feature))
del cluster_feature['exchange58']
del cluster_feature['exchange357']
del cluster_feature['exchange368']
del cluster_feature['exchange384']
print(len(cluster_feature))

571
567


## ----------------------------------------------------------------------------------------------------------------------
## 거래소 분류 모델을 사용한 exchange분류

In [76]:
exchange_feature = dict()
with open('./category_feature_pickle/mining_scam_ransomware_exchange.pickle', 'rb') as fr:
    exchange_feature = pickle.load(fr)

In [49]:
import pickle

bithumb_feature = dict()
with open('/home/dnlabblocksci/바탕화면/backup_bitcoinfile_dongjun/dongjun-blocksci-test/db/four_exchange_feature_file/bithumb_cluster_address_part1.pickle', 'rb') as fr:
    bithumb_feature = pickle.load(fr)

In [50]:
korbit_feature = dict()
with open('/home/dnlabblocksci/바탕화면/backup_bitcoinfile_dongjun/dongjun-blocksci-test/db/four_exchange_feature_file/korbit_cluster_address.pickle', 'rb') as fr:
    korbit_feature = pickle.load(fr)

In [51]:
coinone_feature = dict()
with open('/home/dnlabblocksci/바탕화면/backup_bitcoinfile_dongjun/dongjun-blocksci-test/db/four_exchange_feature_file/coinone_cluster_address.pickle', 'rb') as fr:
    coinone_feature = pickle.load(fr)

In [52]:
upbit_feature = dict()
with open('/home/dnlabblocksci/바탕화면/backup_bitcoinfile_dongjun/dongjun-blocksci-test/db/four_exchange_feature_file/upbit_cluster_address.pickle', 'rb') as fr:
    upbit_feature = pickle.load(fr)

In [53]:
# change [min, max, avg] to variables
def change_dict(feature_dict):
    fixed_feature_dict = dict(); label_list = list()
    feature_name = ['btc_spent', 'btc_received', 'usd_spent', 'usd_received', 'balance', 'age', 'fee', 'number_spent', 'number_received']
    for fn in feature_name:
        fixed_feature_dict[fn + '_max'] = list()
        fixed_feature_dict[fn + '_min'] = list()
        fixed_feature_dict[fn + '_avg'] = list()
    feature_list = list(feature_dict.keys())
    
    for fl in feature_list:
        _body = feature_dict[str(fl)]
        label_list.append(fl)
        for i, b in enumerate(_body):
            fixed_feature_dict[feature_name[i] + '_max'] += [b[0]]
            fixed_feature_dict[feature_name[i] + '_min'] += [b[1]]
            fixed_feature_dict[feature_name[i] + '_avg'] += [b[2]]
    
    return fixed_feature_dict, label_list

In [54]:
fixed_bithumb_feature, bithumb_labels = change_dict(bithumb_feature)
fixed_korbit_feature, korbit_labels = change_dict(korbit_feature)
fixed_coinone_feature, coinone_labels = change_dict(coinone_feature)
fixed_upbit_feature, upbit_labels = change_dict(upbit_feature)

In [55]:
import pandas as pd
bithumb_feature_df = pd.DataFrame(fixed_bithumb_feature)
bithumb_label_df = pd.DataFrame({'labels': bithumb_labels})
bithumb_ = pd.concat([bithumb_label_df, bithumb_feature_df], axis=1)

korbit_feature_df = pd.DataFrame(fixed_korbit_feature)
korbit_label_df = pd.DataFrame({'labels': korbit_labels})
korbit_ = pd.concat([korbit_label_df, korbit_feature_df], axis=1)

coinone_feature_df = pd.DataFrame(fixed_coinone_feature)
coinone_label_df = pd.DataFrame({'labels': coinone_labels})
coinone_ = pd.concat([coinone_label_df, coinone_feature_df], axis=1)

upbit_feature_df = pd.DataFrame(fixed_upbit_feature)
upbit_label_df = pd.DataFrame({'labels': upbit_labels})
upbit_ = pd.concat([upbit_label_df, upbit_feature_df], axis=1)

In [56]:
category_feature_df = pd.concat([bithumb_feature_df, korbit_feature_df, coinone_feature_df, upbit_feature_df])
label_df = pd.concat([bithumb_label_df, korbit_label_df, coinone_label_df, upbit_label_df])
result_for_pca = pd.concat([bithumb_, korbit_, coinone_, upbit_])

In [57]:
category_feature_df = category_feature_df.reset_index().drop(['index'], axis=1)
category_feature_df.head()

,btc_spent_max,btc_spent_min,btc_spent_avg,btc_received_max,btc_received_min,btc_received_avg,usd_spent_max,usd_spent_min,usd_spent_avg,usd_received_max,...,age_avg,fee_max,fee_min,fee_avg,number_spent_max,number_spent_min,number_spent_avg,number_received_max,number_received_min,number_received_avg
0,77798.077159,0.000010,224.505073,20.000,0.000010,0.492947,3.359075e+07,0.005888,9.930458e+04,12760.000000,...,1.034617e+07,77789.077159,0.000000,224.012127,3234,1,14.570806,3202,1,14.753813
1,4212.611001,0.010000,422.016999,19.349,0.000253,2.534915,8.888869e+06,8.720061,6.547492e+05,18082.761762,...,1.485026e+07,4212.592001,0.000000,419.482085,479,1,94.447368,451,1,88.947368
2,7445.180929,0.150670,82.088296,37.000,0.001164,0.863586,3.866849e+07,581.451243,4.630368e+05,89266.200000,...,1.426833e+07,7444.265094,0.000000,81.224711,4367,1,81.602694,7831,1,78.272727
3,4064.163600,4.759344,1027.443665,13.258,0.170386,4.412492,2.447705e+07,29453.959211,7.055855e+06,51812.264000,...,1.209650e+07,4060.080600,1.388324,1023.031173,553,2,151.750000,802,1,95.333333
4,8865.334185,1.449975,548.452785,13.258,0.145840,2.662719,4.627508e+07,8173.945984,3.180038e+06,51812.264000,...,1.140528e+07,8862.881185,0.000000,545.790065,4367,1,167.000000,7831,1,239.452381


### LightGBM 사용

In [71]:
from lightgbm import LGBMClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [61]:
change_number = {'bithumb': 0, 'korbit': 1, 'coinone': 2, 'upbit': 3}

def change_lable_to_num(label):
    for cn in change_number:
        if cn in label:
            return change_number[cn]

In [62]:
label_to_num = label_df.copy()

In [63]:
label_to_num['label_num'] = label_to_num.apply(lambda x:change_lable_to_num(x['labels']), axis=1)
label_to_num = label_to_num.drop(['labels'], axis=1)

In [64]:
label_to_num.astype({'label_num': int})
label_to_num.head()

,label_num
0,0
1,0
2,0
3,0
4,0


In [68]:
X_train, X_test, y_train, y_test = train_test_split(category_feature_df, label_to_num, test_size=0.2, random_state=10)

In [69]:
lgb = LGBMClassifier(n_estimators=400)
lgb.fit(X_train, y_train)

/home/dnlabblocksci/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LGBMClassifier(n_estimators=400)

In [72]:
lgb_pred = lgb.predict(X_test)
print("정확도 : {0:.3f}".format(accuracy_score(y_test, lgb_pred)))

정확도 : 0.953


In [77]:
ex_feature, ex_label = change_dict(exchange_feature)
ex_feature_df = pd.DataFrame(ex_feature)
ex_label_df = pd.DataFrame({'labels': ex_label})

In [85]:
ex_label_df.iloc[146:, :]

,labels
146,exchange0
147,exchange1
148,exchange2
149,exchange3
150,exchange4
...,...
566,exchange421
567,exchange422
568,exchange423
569,exchange424


In [87]:
only_exchange = ex_feature_df.iloc[146:, :] # 146부터가 exchange

In [90]:
predict_ex = lgb.predict(only_exchange)

In [94]:
import collections
collections.Counter(predict_ex)

Counter({1: 257, 0: 93, 2: 61, 3: 14})

In [95]:
pd.DataFrame({'labels' : predict_ex})

,labels
0,1
1,1
2,1
3,1
4,0
...,...
420,1
421,1
422,0
423,0


In [98]:
only_exchange = only_exchange.reset_index().drop(['index'], axis=1)

In [103]:
concat_exchange = pd.concat([pd.DataFrame({'labels' : predict_ex}), only_exchange], axis=1)
concat_exchange.head()

,labels,btc_spent_max,btc_spent_min,btc_spent_avg,btc_received_max,btc_received_min,btc_received_avg,usd_spent_max,usd_spent_min,usd_spent_avg,...,age_avg,fee_max,fee_min,fee_avg,number_spent_max,number_spent_min,number_spent_avg,number_received_max,number_received_min,number_received_avg
0,1,52.042482,8.510636,36.295453,52.042482,8.510636,36.295453,2.663563e+04,5.796361e+03,1.903797e+04,...,3.817507e+07,0.0,-7.105427e-15,-2.368476e-15,1367,6,756.000000,1367,6,756.000000
1,1,0.010000,0.000078,0.003704,0.010000,0.000078,0.003704,1.154915e+01,1.093942e-02,4.251041e+00,...,2.396534e+06,0.0,0.000000e+00,0.000000e+00,2,1,1.333333,2,1,1.333333
2,1,0.016431,0.003489,0.009483,0.016431,0.003489,0.009483,5.675399e+00,1.631303e+00,3.417318e+00,...,4.524307e+07,0.0,0.000000e+00,0.000000e+00,8,1,3.333333,8,1,3.333333
3,1,7.231811,7.231811,7.231811,7.231811,7.231811,7.231811,2.030193e+03,2.030193e+03,2.030193e+03,...,9.729845e+06,0.0,0.000000e+00,0.000000e+00,349,349,349.000000,349,349,349.000000
4,0,406.493830,406.493830,406.493830,406.493830,406.493830,406.493830,4.050691e+06,4.050691e+06,4.050691e+06,...,1.752022e+07,0.0,0.000000e+00,0.000000e+00,79,79,79.000000,79,79,79.000000


In [104]:
concat_exchange.to_csv('./category_feature_pickle/change_to_four_exchange.csv')